In [4]:
from os import listdir
import numpy as np
import matplotlib.pyplot as plt

In [7]:
# acquire all file names in directory
filespath = '/Users/samuelfogarty/OneDrive/Desktop/OneDrive/photondetectortests/dune_fd_pds/csv/singlephotonsearch/'
abovebreakdown = '12usecwindow__25MHzSR_1kpts_10mVperdiv_ch4_44.135Bias'
belowbreakdown = '12usecwindow_25MHzSR_1kpts_10mVperdiv_ch4_10.932VBias'

filespath_bbd = filespath + belowbreakdown
filespath_abd = filespath + abovebreakdown
filenames_bbd = os.listdir(filespath_bbd)
filenames_abd = os.listdir(filespath_abd)

In [ ]:
times = []
voltages = []
with open((filenames_bbd + filenames_bbd[0])) as fp: # this line opens the file and makes sure to close it when all is done
        line = fp.readline()
        while line: # start read file loop
            word = line.strip().split(',') # splits each line at a comma
            if word[0] != '':
                bincenters.append(float(word[0]))
                bincontents.append(float(word[1]))
            line = fp.readline()
bincenters = np.array(bincenters)*1e9
bincontents = np.array(bincontents)

In [18]:
def readfile(filepath):
    word,words = 0,0
    ts, Vs1 = [], [] # seconds, volts


    with open((filespath_bbd + '/' + filenames_bbd[0])) as fp: # this line opens the file and makes sure to close it when all is done
        line = fp.readline()
        while line: # start read file loop
            words = line.strip().split(',') # splits each line at a comma
                                                # the strip part makes sure there aren't any weird characters left over at the start or end of a line
            if (words[0] == 'TIME' and words[0] != ''): # finds where to start taking the data, may need to change 
                word = 1                                # for different files
                line = fp.readline() # get next line
                words = line.strip().split(',')
            if word == 1 and words[0] != '': # this ensures it starts recording the data lines only 
                ts.append(words[0].strip())
                Vs1.append(words[1].strip()) 
            line = fp.readline()
    return ts, Vs1
